In [ ]:
# prac 3
# Pow

# scroll down to next code block for bfs

import hashlib
import time

class Block:
    def __init__(self, index, previous_hash, transactions, difficulty, reward):
        self.index = index
        self.previous_hash = previous_hash
        self.transactions = transactions
        self.timestamp = time.time()
        self.nonce = 0
        self.difficulty = difficulty
        self.reward = reward
        self.hash = self.mine_block()

    def compute_hash(self):
        block_data = f"{self.index}{self.previous_hash}{self.transactions}{self.timestamp}{self.nonce}"
        return hashlib.sha256(block_data.encode()).hexdigest()

    def mine_block(self):
        target = "0" * self.difficulty
        while True:
            self.hash = self.compute_hash()
            if self.hash.startswith(target):
                return self.hash
            self.nonce += 1

class Blockchain:
    def __init__(self, difficulty=4, initial_reward=50, halving_interval=5):
        self.chain = []
        self.difficulty = difficulty
        self.block_reward = initial_reward
        self.halving_interval = halving_interval
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, "0", "Genesis Block", self.difficulty, self.block_reward)
        self.chain.append(genesis_block)

    def add_block(self, transactions):
        previous_block = self.chain[-1]
        new_block = Block(len(self.chain), previous_block.hash, transactions, self.difficulty, self.block_reward)
        self.chain.append(new_block)

        # Simulate reward halving
        if len(self.chain) % self.halving_interval == 0:
            self.block_reward /= 2

        self.adjust_difficulty()

    def adjust_difficulty(self):
        if len(self.chain) > 1:
            time_taken = self.chain[-1].timestamp - self.chain[-2].timestamp
            if time_taken > 5:  #
                self.difficulty -= 1 if self.difficulty > 1 else 0
            elif time_taken < 2:
                self.difficulty += 1

    def display_chain(self):
        for block in self.chain:
            print(f"Block {block.index}:")
            print(f"  Hash: {block.hash}")
            print(f"  Prev: {block.previous_hash}")
            print(f"  Transactions: {block.transactions}")
            print(f"  Nonce: {block.nonce}")
            print(f"  Reward: {block.reward}")
            print(f"  Difficulty: {block.difficulty}")
            print("-" * 50)

blockchain = Blockchain()

transactions_list = ["Aryana -> Anvesha: 5 BTC", "Kinjal -> Gaurang: 3 BTC"]

for _ in range(10):
    blockchain.add_block(transactions_list)

blockchain.display_chain()

In [ ]:
# To study the concept of multithreading and implement Byzantine Generals Problem in any language.

# prac 4
import threading
import random
import time
import hashlib

class Block:
    def __init__(self, general_id, message, previous_hash=''):
        self.general_id = general_id
        self.message = message
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        data = f"{self.general_id}{self.message}{self.previous_hash}"
        return hashlib.sha256(data.encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(general_id="Genesis", message="Start", previous_hash="0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]

            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

def general_action(general_id, message, shared_data, lock):
    time.sleep(random.uniform(0.1, 0.5))

    with lock:
        print(f"General {general_id} sends message: {message}")
        new_block = Block(general_id, message)
        shared_data['blockchain'].add_block(new_block)

        if len(shared_data['blockchain'].chain) - 1 == shared_data['total_generals']:
            tally_votes(shared_data)

def tally_votes(shared_data):
    blockchain = shared_data['blockchain']
    votes = [block.message for block in blockchain.chain[1:]]

    decision = max(set(votes), key=votes.count)

    print("\nBlockchain Contents:")
    for block in blockchain.chain:
        print(f"General {block.general_id}: {block.message} (Hash: {block.hash[:8]})")

    print(f"\nConsensus reached: {decision}\n")
    print(f"Blockchain Validity: {blockchain.is_chain_valid()}")

def byzantine_generals_simulation(total_generals, malicious_generals):
    shared_data = {
        'blockchain': Blockchain(),
        'total_generals': total_generals
    }
    lock = threading.Lock()
    threads = []

    for i in range(total_generals):
        if i < malicious_generals:
            message = "Attack" if random.random() < 0.5 else "Retreat"
        else:
            # Loyal generals send consistent messages
            message = "Attack"

        t = threading.Thread(target=general_action, args=(i, message, shared_data, lock))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

if __name__ == "__main__":
    total_generals = 5
    malicious_generals = 2
    byzantine_generals_simulation(total_generals, malicious_generals)